# Baseline

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from torch.utils.tensorboard import SummaryWriter

import numpy as np

from sklearn.metrics import roc_auc_score, f1_score

from pathlib import Path
from typing import Tuple
import pandas as pd

from tqdm import tqdm
from functools import partial

tqdm = partial(tqdm, position=0, leave=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
EMBBEDDINGS_PATH = Path("embbeddings")
DATA_PATH = Path("data")
LOG_DIR = Path("logs")
RESULTS_DIR = Path("results")
CHECKPOINTS_DATA = Path("checkpoints")
RANDOM_STATE = 345

NUM_CLASSES = 37

CHECKPOINTS_DATA.mkdir(exist_ok=True)

In [3]:
%load_ext tensorboard
%tensorboard --logdir $LOG_DIR

Reusing TensorBoard on port 6006 (pid 5822), started 1:52:27 ago. (Use '!kill 5822' to kill it.)

In [4]:
tensorboard_writer = SummaryWriter(LOG_DIR)

## Przygotowanie zbioru i datasetu

In [5]:
class EmailSimpleDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(
        self,
        embbedings_size: int,
    ):
        """
        Args:
            df (string): DataFrame with network data
        """
        
        if embbedings_size != 64 and embbedings_size != 128:
            raise ValueError("Only embbedings with size 64 and 128 are implemented!")
    
        self.df = pd.read_csv(DATA_PATH / f"train_data_{embbedings_size}.tsv", sep=" ")
        self.embbeddings = KeyedVectors.load_word2vec_format(
            EMBBEDDINGS_PATH / f"embbeddings_{embbedings_size}.graphvectors",
            binary=False
        )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        embbedding_id = self.df.iloc[idx].embbedding_id
        embbedding = self.embbeddings[embbedding_id] 
        
        department = self.df.iloc[idx].department_id
        embbedding = torch.from_numpy(embbedding)
        return (embbedding, int(department))

In [31]:
class Baseline(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        num_classes: int,
        name: str = "baseline"
    ):
        super().__init__()
        self.name = name
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, num_classes)
        self.soft = nn.Softmax(dim=1)

    def predict(self, x):
        with torch.no_grad():
            return self(x).argmax(dim=1)

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.soft(self.linear2(x))
        return x

In [7]:
def prepare_dataset(
    embbeddings_size: int,
    split_val: float = 0.5,
    batch_size: int = 8
) -> Tuple[DataLoader, DataLoader, Subset, Subset]:

    dataset = EmailSimpleDataset(embbeddings_size)
    df = dataset.df
    train_ids, test_ids = train_test_split(
        np.arange(len(dataset)),
        stratify=np.array(dataset.df.department_id),
        test_size=split_val,
        random_state=RANDOM_STATE
    )
    test_ids, val_ids = train_test_split(
        test_ids,
        stratify=np.array([df.iloc[y]["department_id"] for y in test_ids]),
        test_size=0.5,
        random_state=RANDOM_STATE
    )
    
    
    train_ds, test_ds = Subset(dataset, train_ids), Subset(dataset, test_ids)
    val_ds = Subset(dataset, val_ids)
    
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_dl = DataLoader(val_ds, batch_size=batch_size, pin_memory=True)
    test_dl = DataLoader(val_ds, batch_size=batch_size, pin_memory=True)
    
    return train_dl, test_dl, val_dl

In [8]:
def validate(
    model: torch.nn.Module, 
    loss_fn: torch.nn.CrossEntropyLoss, 
    dataloader: DataLoader
) -> Tuple[torch.Tensor, torch.Tensor, np.array]:
    loss = 0
    all_vals = 0
    aucs = 0
    target = []
    preds = []
    preds_vec = []
    
    for X_batch, y_batch in dataloader:
        y_pred = model(X_batch.to(device))
        all_vals += len(y_pred)
        loss += loss_fn(y_pred, y_batch.to(device)).sum()
        target.extend(y_batch.cpu().numpy())
        preds.extend(torch.argmax(y_pred, dim=1).cpu().numpy())
        preds_vec.append(y_pred.squeeze().cpu().numpy())
    return (
        loss / all_vals,
        f1_score(target, preds, average="weighted"),
        roc_auc_score(y_true=target, y_score=np.concatenate(preds_vec, axis=0), multi_class="ovr")
    )

In [9]:
def fit(
    model: torch.nn.Module,
    optimiser: torch.optim.Optimizer, 
    loss_fn: torch.nn.CrossEntropyLoss, 
    train_dl: DataLoader, 
    val_dl: DataLoader, 
    print_metrics: str = True,
    epochs: int = 10
):
    prev_f1 = 0
    for epoch in tqdm(range(epochs)):
        best = False
        for X_batch, y_batch in tqdm(train_dl):
            y_pred = model(X_batch.to(device))
            loss = loss_fn(y_pred, y_batch.to(device))
            loss.backward()
            optimiser.step()
            optimiser.zero_grad()

        if print_metrics: 
            model.eval()
            with torch.no_grad():
                
                train_loss, train_f1, train_auc = validate(
                  model=model, loss_fn=loss_fn, dataloader=train_dl
                ) 
                val_loss, val_f1, val_auc = validate(
                  model=model, loss_fn=loss_fn, dataloader=val_dl
                )
                
                tensorboard_writer.add_scalars(
                    main_tag=f'{model.name}_loss', 
                    tag_scalar_dict={
                        'train': train_loss,
                        'dev': val_loss
                    }, 
                    global_step=epoch+1
                )
                
                tensorboard_writer.add_scalars(
                    main_tag=f'{model.name}_auc', 
                    tag_scalar_dict={
                        'train': train_auc,
                        'dev': val_auc
                    }, 
                    global_step=epoch+1
                )
                
                tensorboard_writer.add_scalars(
                    main_tag=f'{model.name}_f1', 
                    tag_scalar_dict={
                        'train': train_f1,
                        'dev': val_f1
                    }, 
                    global_step=epoch+1
                )
                if prev_f1 < val_f1:
                    best = True
                    prev_f1 = val_f1
                    torch.save(model.state_dict(), CHECKPOINTS_DATA / f"{model.name}.pt")

                print(
                  f"Epoch {epoch}:  (best: {best})"
                  f"train loss = {train_loss:.3f} (auc: {train_auc:.3f} f1: {train_f1:.3f}), "
                  f"validation loss = {val_loss:.3f} (auc: {val_auc:.3f} f1: {val_f1:.3f})"
                )
    return model

## Baseline dla wektorów osadzeń o rozmiarze 64

In [17]:
hparams = {
    "num_epochs": 20,
    "hidden_dim":  32,
    "emb_dim": 64,
    "lr": 1e-2,
    "model_name": "Baseline_64"
}

In [18]:
train_dl, test_dl, val_dl = prepare_dataset(
    embbeddings_size=64,
    split_val=0.5,
    batch_size=32
)

baseline = Baseline(
    in_dim=hparams["emb_dim"],
    hidden_dim=hparams["hidden_dim"],
    num_classes=NUM_CLASSES,
    name=hparams["model_name"]
    
).to(device)

optimiser = torch.optim.Adam(baseline.parameters(), lr=hparams["lr"])
loss_func = torch.nn.CrossEntropyLoss()

baseline = fit(
    baseline,
    optimiser,
    loss_func,
    train_dl,
    test_dl,
    epochs=hparams["num_epochs"],
    print_metrics=True
)

  5%|██▏                                         | 1/20 [00:00<00:06,  3.17it/s]

Epoch 0:  (best: True)train loss = 0.114 (auc: 0.851 f1: 0.100), validation loss = 0.115 (auc: 0.838 f1: 0.094)


 10%|████▍                                       | 2/20 [00:00<00:05,  3.43it/s]

Epoch 1:  (best: True)train loss = 0.108 (auc: 0.842 f1: 0.264), validation loss = 0.109 (auc: 0.851 f1: 0.225)


 15%|██████▌                                     | 3/20 [00:00<00:05,  3.35it/s]

...

Epoch 17:  (best: False)train loss = 0.096 (auc: 0.902 f1: 0.635), validation loss = 0.098 (auc: 0.897 f1: 0.582)


 95%|████████████████████████████████████████▊  | 19/20 [00:05<00:00,  3.76it/s]

Epoch 18:  (best: True)train loss = 0.096 (auc: 0.909 f1: 0.676), validation loss = 0.098 (auc: 0.900 f1: 0.608)


100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.68it/s]

Epoch 19:  (best: False)train loss = 0.095 (auc: 0.905 f1: 0.663), validation loss = 0.098 (auc: 0.901 f1: 0.586)


In [19]:
with torch.no_grad():
    test_loss, test_f1, test_auc = validate(baseline, loss_func, test_dl)

In [20]:
print(f"""
          Test loss: {test_loss},
          Test F1: {test_f1},
          Test AUC: {test_auc}
      """)


          Test loss: 0.09814397245645523,
          Test F1: 0.586444934334713,
          Test AUC: 0.9008375183180345
      


## Baseline dla wektorów osadzeń o rozmiarze 128

In [27]:
hparams = {
    "num_epochs": 30,
    "hidden_dim":  32,
    "emb_dim": 128,
    "lr": 5e-3,
    "model_name": "Baseline_128"
}

In [28]:
train_dl, test_dl, val_dl = prepare_dataset(
    embbeddings_size=128,
    split_val=0.5,
    batch_size=32
)

baseline = Baseline(
    in_dim=hparams["emb_dim"],
    hidden_dim=hparams["hidden_dim"],
    num_classes=NUM_CLASSES,
    name=hparams["model_name"]
    
).to(device)

optimiser = torch.optim.Adam(baseline.parameters(), lr=hparams["lr"])
loss_func = torch.nn.CrossEntropyLoss()

baseline = fit(
    baseline,
    optimiser,
    loss_func,
    train_dl,
    test_dl,
    epochs=hparams["num_epochs"],
    print_metrics=True
)

  3%|█▍                                          | 1/30 [00:00<00:07,  3.78it/s]

Epoch 0:  (best: True)train loss = 0.117 (auc: 0.891 f1: 0.194), validation loss = 0.117 (auc: 0.824 f1: 0.175)


  7%|██▉                                         | 2/30 [00:00<00:07,  3.76it/s]

Epoch 1:  (best: False)train loss = 0.111 (auc: 0.802 f1: 0.153), validation loss = 0.112 (auc: 0.794 f1: 0.148)


 10%|████▍                                       | 3/30 [00:00<00:07,  3.73it/s]

...

Epoch 27:  (best: False)train loss = 0.096 (auc: 0.903 f1: 0.654), validation loss = 0.098 (auc: 0.891 f1: 0.596)


 97%|█████████████████████████████████████████▌ | 29/30 [00:07<00:00,  3.82it/s]

Epoch 28:  (best: False)train loss = 0.095 (auc: 0.903 f1: 0.654), validation loss = 0.098 (auc: 0.891 f1: 0.593)


100%|███████████████████████████████████████████| 30/30 [00:08<00:00,  3.75it/s]

Epoch 29:  (best: False)train loss = 0.096 (auc: 0.903 f1: 0.653), validation loss = 0.098 (auc: 0.891 f1: 0.593)


In [29]:
with torch.no_grad():
    test_loss, test_f1, test_auc = validate(baseline, loss_func, test_dl)

In [30]:
print(f"""
          Test loss: {test_loss},
          Test F1: {test_f1},
          Test AUC: {test_auc}
      """)


          Test loss: 0.09844127297401428,
          Test F1: 0.5930532099677123,
          Test AUC: 0.8906585195034016
      


In [32]:
data = {
    "test_auc": [],
    "test_f1": [],
    "model_name": [],
}

hparams = {
    "num_epochs": 20,
    "hidden_dim":  32,
    "lr": 1e-2,
}
    
for embbeding_size in [64, 128]:
    
    hparams["model_name"] = f"Baseline_{embbeding_size}"
    hparams["emb_dim"] = embbeding_size
    
    for _ in range(10):
        train_dl, test_dl, val_dl = prepare_dataset(
            embbeddings_size=embbeding_size,
            split_val=0.5,
            batch_size=32
        )

        baseline = Baseline(
            in_dim=hparams["emb_dim"],
            hidden_dim=hparams["hidden_dim"],
            num_classes=NUM_CLASSES,
            name=hparams["model_name"]

        ).to(device)

        optimiser = torch.optim.Adam(baseline.parameters(), lr=hparams["lr"])
        loss_func = torch.nn.CrossEntropyLoss()

        baseline = fit(
            baseline,
            optimiser,
            loss_func,
            train_dl,
            test_dl,
            epochs=hparams["num_epochs"],
            print_metrics=True
        )
        with torch.no_grad():
            test_loss, test_f1, test_auc = validate(baseline, loss_func, test_dl)
        data["test_f1"].append(test_f1)
        data["test_auc"].append(test_auc)
        data["model_name"].append(hparams["model_name"])


  5%|██▏                                         | 1/20 [00:00<00:06,  3.02it/s]

Epoch 0:  (best: True)train loss = 0.115 (auc: 0.849 f1: 0.194), validation loss = 0.115 (auc: 0.836 f1: 0.171)


 10%|████▍                                       | 2/20 [00:00<00:05,  3.24it/s]

Epoch 1:  (best: True)train loss = 0.107 (auc: 0.844 f1: 0.387), validation loss = 0.108 (auc: 0.846 f1: 0.355)


 15%|██████▌                                     | 3/20 [00:00<00:04,  3.46it/s]

...

Epoch 17:  (best: True)train loss = 0.095 (auc: 0.907 f1: 0.694), validation loss = 0.098 (auc: 0.892 f1: 0.618)


 95%|████████████████████████████████████████▊  | 19/20 [00:04<00:00,  3.93it/s]

Epoch 18:  (best: False)train loss = 0.095 (auc: 0.910 f1: 0.697), validation loss = 0.098 (auc: 0.890 f1: 0.616)


100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.89it/s]


Epoch 19:  (best: True)train loss = 0.095 (auc: 0.917 f1: 0.707), validation loss = 0.098 (auc: 0.893 f1: 0.625)


In [33]:
pd.DataFrame(data).to_csv(RESULTS_DIR / "baseline_stats.csv", index=False)